In [27]:
import streamlit as st
import numpy as np
import pandas as pd
from PIL import Image
import gspread
import json
import pandas as pd
#ServiceAccountCredentials：Googleの各サービスへアクセスできるservice変数を生成します。
from oauth2client.service_account import ServiceAccountCredentials

#2つのAPIを記述しないとリフレッシュトークンを3600秒毎に発行し続けなければならない
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
json_path = 'kitaichi-fd2dd716665b.json'
#認証情報設定
#ダウンロードしたjsonファイル名をクレデンシャル変数に設定（秘密鍵、Pythonファイルから読み込みしやすい位置に置く）
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_path, scope)
#OAuth2の資格情報を使用してGoogle APIにログインします。
gc = gspread.authorize(credentials)
# １．ファイル名を指定してワークブックを選択
workbook = gc.open('収支表')
sh_secchi = workbook.worksheet('設置一覧')
sh_shop = workbook.worksheet('店一覧')
sh_slot_kisyu = workbook.worksheet('スロット機種一覧')
df_secchi = pd.DataFrame(sh_secchi.get_all_values()[1:], columns=sh_secchi.get_all_values()[0])
df_shop = pd.DataFrame(sh_shop.get_all_values()[1:], columns=sh_shop.get_all_values()[0])
df_slot_kisyu = pd.DataFrame(sh_slot_kisyu.get_all_values()[1:], columns=sh_slot_kisyu.get_all_values()[0])

In [18]:
df_secchi

,店番号,台番号,機種番号
0,shop_01,1,s_0001
1,shop_01,2,s_0002
2,shop_01,3,s_0003
3,shop_01,4,s_0004
4,shop_01,5,s_0005
...,...,...,...
195,shop_01,236,s_0092
196,shop_01,237,s_0093
197,shop_01,238,s_0094
198,shop_01,239,s_0095


In [6]:
list(df_secchi['店番号'])

['shop_01']

In [9]:
list(df_secchi['店名'])

['ダイナム宇部港町店']

In [14]:
target_shop_name = 'ダイナム宇部港町店'
list(df_secchi['店番号'])[(list(df_secchi['店名']).index(target_shop_name))]

'shop_01'

In [13]:
target_shop_name = 'ダイナム宇部港町店'
list(df_secchi['店名']).index(target_shop_name)

0

In [15]:
df_slot_kisyu.head()

,機種番号,機種名,型名,メーカー,タイプ,コイン持ち,天井,純増,リセット恩恵,リセット判別,やめ時
0,s_0001,CREA Newクレアの秘宝伝,Ｓ ＣＲＥＡ～クレアの秘宝～Ａ２,大都技研,A,39,なし,なし,なし,,
1,s_0002,もっと!クレアの秘宝伝 女神の歌声と太陽の子供達,Ｓもっと！クレアの秘宝伝ＰＡ２,パオン・ディーピー,A+RT,39,なし,なし,なし,,
2,s_0003,CCエンジェル,Ｓ／ＣＣエンジェル／ＣＡ,メーシー,A+RT,42,なし,0.2,なし,,
3,s_0004,Re:ゼロから始める異世界生活Apex Vacation,ＳリゼロＡｐｅｘＶ Ａ６,サボハニ,A+AT,40,なし,なし,なし,,
4,s_0005,ケロット4,Ｓケロット４ＣＣ,セブンリーグ,A,40,なし,なし,なし,,


In [16]:
df_slot_kisyu[df_slot_kisyu['機種番号'] =='s_0005']

,機種番号,機種名,型名,メーカー,タイプ,コイン持ち,天井,純増,リセット恩恵,リセット判別,やめ時
4,s_0005,ケロット4,Ｓケロット４ＣＣ,セブンリーグ,A,40,なし,なし,なし,,


In [ ]:
tuple(df_secchi[df_secchi['店番号'] =='shop_01']['台番号'])

In [29]:
df_slot_kisyu.head()

,機種番号,機種名,型名,メーカー,タイプ,コイン持ち,天井,純増,リセット恩恵,リセット判別,やめ時
0,s_0001,CREA Newクレアの秘宝伝,Ｓ ＣＲＥＡ～クレアの秘宝～Ａ２,大都技研,A,39,なし,なし,なし,,
1,s_0002,もっと!クレアの秘宝伝 女神の歌声と太陽の子供達,Ｓもっと！クレアの秘宝伝ＰＡ２,パオン・ディーピー,A+RT,39,なし,なし,なし,,
2,s_0003,CCエンジェル,Ｓ／ＣＣエンジェル／ＣＡ,メーシー,A+RT,42,なし,0.2,なし,,
3,s_0004,Re:ゼロから始める異世界生活Apex Vacation,ＳリゼロＡｐｅｘＶ Ａ６,サボハニ,A+AT,40,なし,なし,なし,,
4,s_0005,ケロット4,Ｓケロット４ＣＣ,セブンリーグ,A,40,なし,なし,なし,,


In [32]:
list_katamei = list(df_slot_kisyu['型名'])

In [87]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())

In [97]:
list_kisyumei = []
for katamei in tqdm(list_katamei):
    driver.get('https://www.google.com/')
    search = driver.find_element_by_name('q')   # HTML内で検索ボックス(name='q')を指定する
    search.send_keys(katamei.replace('\t','') + ' pworld')             # 検索ワードを送信する
    search.submit()
    kensaku_ichiran = driver.find_elements_by_class_name('yuRUbf')
    for kekka in kensaku_ichiran:
        url = kekka.find_elements_by_tag_name('a')[0].get_attribute('href')
        if 'p-world.co.jp/machine/database/' in url:
            driver.get(url)
            kisyu_name = driver.find_element_by_class_name('contensTitle').text
            list_kisyumei.append(kisyu_name)
            break

100%|██████████| 96/96 [11:46<00:00,  7.36s/it]


In [98]:
print(len(list(df_slot_kisyu['型名'])))
print(len(list_kisyumei))

96
96


In [99]:
df_slot_kisyu['機種名']=list_kisyumei

In [100]:
df_slot_kisyu

,機種番号,機種名,型名,メーカー,タイプ,コイン持ち,天井,純増,リセット恩恵,リセット判別,やめ時
0,s_0001,CREA Newクレアの秘宝伝,Ｓ ＣＲＥＡ～クレアの秘宝～Ａ２,大都技研,A,39,なし,なし,なし,,
1,s_0002,もっと!クレアの秘宝伝 女神の歌声と太陽の子供達,Ｓもっと！クレアの秘宝伝ＰＡ２,パオン・ディーピー,A+RT,39,なし,なし,なし,,
2,s_0003,CCエンジェル,Ｓ／ＣＣエンジェル／ＣＡ,メーシー,A+RT,42,なし,0.2,なし,,
3,s_0004,Re:ゼロから始める異世界生活Apex Vacation,ＳリゼロＡｐｅｘＶ Ａ６,サボハニ,A+AT,40,なし,なし,なし,,
4,s_0005,パチスロ ケロット4,Ｓケロット４ＣＣ,セブンリーグ,A,40,なし,なし,なし,,
...,...,...,...,...,...,...,...,...,...,...,...
91,s_0092,SLOTえとたま,Ｓ／えとたま／ＥＭ,メーシー,AT,37,800,2,,できない,
92,s_0093,アカメが斬る!,ＳＡＴアカメが斬る！Ｋ,七匠,AT,50,999,4,,できない,
93,s_0094,パチスロANEMONE 交響詩篇エウレカセブン HI‐EVOLUTION,Ｓ ＡＮＥＭＯＮＥ ＦＦ,サミー,AT,37,808,2.6,,,
94,s_0095,パチスロ鉄拳4デビルVer.,Ｓ鉄拳４デビルバージョンＳＬＦＦ,山佐,AT,50,756,2.7,あり,,


In [101]:
from gspread_dataframe import set_with_dataframe
set_with_dataframe(sh_slot_kisyu, df_slot_kisyu,include_index = False)